# RFM Analysis an extension of Customer segmentation.

- RFM is a method used for analyzing customer value. 
- It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

### RFM stands for the three dimensions:

- Recency – How recently did the customer purchase?
- Frequency – How often do they purchase?
- Monetary Value – How much do they spend?

In [79]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [82]:
#importing data file
orders = pd.read_csv('/home/vivek/Desktop/Spring Project/rfm analsis/sample-data.csv',encoding= 'unicode_escape')

In [83]:
orders.head()

order_date        order_id          customer  grand_total
0   9/7/2011  CA-2011-100006       Dennis Kane        378.0
1   7/8/2011  CA-2011-100090        Ed Braxton        699.0
2  3/14/2011  CA-2011-100293  Neil Franz?sisch         91.0
3  1/29/2011  CA-2011-100328   Jasper Cacioppo          4.0
4   4/8/2011  CA-2011-100363       Jim Mitchum         21.0

##  Making the RFM Table



Set this date to the current day and extract all orders until yesterday.

In [103]:
import datetime as dt
NOW = dt.datetime(2014,12,31)

In [104]:
# Make the date_placed column datetime
orders['order_date'] = pd.to_datetime(orders['order_date'])

Create the RFM Table

In [105]:
#calculating RFM table using lambda function
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()).days, # calculation Recency
                                        'order_id': lambda x: len(x),      # calculation Frequency
                                        'grand_total': lambda x: x.sum()}) # calculation Monetary Value

rfmTable['order_date'] = rfmTable['order_date'].astype(int)
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

## Validating the RFM Table

In [89]:
rfmTable.head()

recency  frequency  monetary_value
customer                                           
Aaron Bergman        415          3           887.0
Aaron Hawkins         12          7          1744.0
Aaron Smayling        88          7          3050.0
Adam Bellavance       54          8          7756.0
Adam Hart             34         10          3249.0

Customer **Aaron Bergman** has 
- frequency:"3" 
- monetary value:"$887" 
- recency:"415 days"

In [90]:
aaron = orders[orders['customer']=='Aaron Bergman']
aaron

order_date        order_id       customer  grand_total
624  2011-02-19  CA-2011-152905  Aaron Bergman         13.0
665  2011-03-07  CA-2011-156587  Aaron Bergman        310.0
2336 2013-11-11  CA-2013-140935  Aaron Bergman        564.0

Inserting the date of Aaron purchase and comparing it to the recency in the rfmTable we verify our RFM table is correct.

In [106]:
(NOW - dt.datetime(2013,11,11)).days==415

True

## Determining RFM Quartiles

In [92]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])

In [93]:
quantiles

recency  frequency  monetary_value
0.25     30.0        5.0          1145.0
0.50     75.0        6.0          2257.0
0.75    183.0        8.0          3784.0

Send quantiles to a dictionary, easier to use.

In [94]:
quantiles = quantiles.to_dict()

In [95]:
quantiles

{'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0},
 'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0}}

## Creating the RFM segmentation table

In [96]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good. 

In [107]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1


In [108]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [109]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [110]:
rfmSegmentation.head()

recency  frequency  monetary_value  R_Quartile  F_Quartile  \
customer                                                                      
Aaron Bergman        415          3           887.0           4           4   
Aaron Hawkins         12          7          1744.0           1           2   
Aaron Smayling        88          7          3050.0           3           2   
Adam Bellavance       54          8          7756.0           2           2   
Adam Hart             34         10          3249.0           2           1   

                 M_Quartile RFMClass  
customer                              
Aaron Bergman             4      444  
Aaron Hawkins             3      123  
Aaron Smayling            2      322  
Adam Bellavance           1      221  
Adam Hart                 2      212

In [111]:

# Uncomment any of the following lines to: copy data to clipboard or save it to a CSV file.
# rfmSegmentation.to_clipboard()
# rfmSegmentation.to_csv('rfm-table.csv', sep=',')

# Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently

In [112]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head(5)

recency  frequency  monetary_value  R_Quartile  F_Quartile  \
customer                                                                     
Sanjit Engle          9         11         12210.0           1           1   
John Lee             21         11          9801.0           1           1   
Pete Kriz             9         12          8647.0           1           1   
Harry Marie           2         10          8237.0           1           1   
Lena Creighton       16         12          7661.0           1           1   

                M_Quartile RFMClass  
customer                             
Sanjit Engle             1      111  
John Lee                 1      111  
Pete Kriz                1      111  
Harry Marie              1      111  
Lena Creighton           1      111